In [1]:
import random
random.seed(42)
import numpy as np
import pandas as pd

from scipy.stats import invwishart
from numpy import sum, sqrt, outer, transpose, diag
from numpy.random import multivariate_normal
from scipy.sparse import coo_matrix
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rating = pd.read_csv("rating.csv")
rating = rating.sample(10000)
rating.describe()

,userId,movieId,rating
count,10000.000000,10000.000000,10000.00000
mean,68389.165900,8678.463000,3.52840
std,39919.767731,19234.649081,1.05435
min,31.000000,1.000000,0.50000
25%,34017.500000,899.000000,3.00000
50%,68111.500000,2139.000000,3.50000
75%,102496.750000,4617.750000,4.00000
max,138484.000000,125990.000000,5.00000


In [3]:
rating_matrix = rating.pivot(index='userId', columns='movieId', values='rating')
rating_matrix = rating_matrix.fillna(0)
R = rating_matrix.to_numpy()

### R = U^T*V   ->   R = U^TBV

In [4]:
# hyperparams: \Theta_0 = {\mu_0, T_0, \nu_0, S_0, sigma}
# \Theta_U = {\mu_U, \Sigma_U}
# \Theta_V = {\mu_V, \Sigma_V}
# \Theta_B = {\mu_B, \Sigma_B}

# Sample \Theta_U, \Theta_V, \Theta_B
# Sample U, V, B
# Compute R

In [5]:
# Initialization
M = rating_matrix.shape[0] # number of customers/users
N = rating_matrix.shape[1] # number of movies
D = 10 # number of latent features
U = np.ones((D, M))
V = np.ones((D, N))
diag_b = range(1, D + 1)
B = diag(diag_b)

mu_0, T_0 = np.zeros((D, 1)), np.eye(D)
nu_0, S_0 = D, np.eye(D)
sigma = 1

In [6]:
nonzero_indices = R.nonzero()
nonzero_values = R[nonzero_indices]

In [7]:
iters = 100

Sigma_u, Sigma_v, Sigma_b = np.eye(D), np.eye(D), np.eye(D)

for it in range(iters):
    # Sample \Theta_U
    mu_ustar = inv(inv(T_0) + M*inv(Sigma_u))@(inv(T_0)@mu_0 + inv(Sigma_u)@sum(U, axis=1, keepdims=True))
    T_ustar = inv(inv(T_0) + M*inv(Sigma_u))
    mu_u = multivariate_normal(np.squeeze(np.asarray(transpose(mu_ustar))), T_ustar, 1)
    
    nu_ustar = nu_0 + M
    S_ustar = S_0 + (U - transpose(mu_u))@transpose(U - transpose(mu_u))
    inv_wishart = invwishart(nu_ustar, S_ustar)
    Sigma_u = inv_wishart.rvs()
    
    # Sample \Theta_V
    mu_vstar = inv(inv(T_0) + N*inv(Sigma_v))@(inv(T_0)@mu_0 + inv(Sigma_v)@sum(V, axis=1, keepdims=True))
    T_vstar = inv(inv(T_0) + N*inv(Sigma_v))
    mu_v = multivariate_normal(np.squeeze(np.asarray(transpose(mu_vstar))), T_vstar, 1)
    
    nu_vstar = nu_0 + N
    S_vstar = S_0 + (V - transpose(mu_v))@transpose(V - transpose(mu_v))
    inv_wishart = invwishart(nu_vstar, S_vstar)
    Sigma_v = inv_wishart.rvs()
    
    # Sample \Theta_B
    mu_bstar = inv(inv(T_0) + 1*inv(Sigma_b))@(inv(T_0)@mu_0 + inv(Sigma_b)@sum(B, axis=1, keepdims=True))
    T_bstar = inv(inv(T_0) + 1*inv(Sigma_b))
    mu_b = multivariate_normal(np.squeeze(np.asarray(transpose(mu_bstar))), T_bstar, 1)
    
    nu_bstar = nu_0 + 1
    S_bstar = S_0 + outer(diag_b - mu_b, diag_b - mu_b)
    inv_wishart = invwishart(nu_bstar, S_bstar)
    Sigma_b = inv_wishart.rvs()
    
    # Sample U
    weighted_V = B@V
    for i in range(M):
        Lambda_ustar = inv(Sigma_u)
        theta_ustar = transpose(inv(Sigma_u)@transpose(mu_u))
        for j in range(N):
            if R[i, j] == 0:
                continue
            V_j = transpose(weighted_V[:, j])
            Lambda_ustar += outer(V_j, V_j)/sigma
            theta_ustar += V_j*R[i, j]/sigma
        theta_ustar = inv(Lambda_ustar)@transpose(theta_ustar)
        U[:, i] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_ustar))), inv(Lambda_ustar), 1)
        
    # Sample V
    weighted_U = transpose(transpose(U)@B)
    for j in range(N):
        Lambda_vstar = inv(Sigma_v)
        theta_vstar = transpose(inv(Sigma_v)@transpose(mu_v))
        for i in range(M):
            if R[i, j] == 0:
                continue
            U_i = transpose(weighted_U[:, i])
            Lambda_vstar += outer(U_i, U_i)/sigma
            theta_vstar += U_i*R[i, j]/sigma
        theta_vstar = inv(Lambda_vstar)@transpose(theta_vstar)
        V[:, j] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_vstar))), inv(Lambda_vstar), 1)
    
    # Sample B
    Lambda_bstar = inv(Sigma_b)
    theta_bstar = transpose(inv(Sigma_b)@transpose(mu_b))
    for k in range(len(nonzero_indices[0])):
        i = nonzero_indices[0][k]
        j = nonzero_indices[1][k]
        A = U[:, i] * V[:, j]
        Lambda_bstar += outer(A, A)/sigma
        theta_bstar += A*R[i, j]/sigma
    theta_bstar = inv(Lambda_bstar)@transpose(theta_bstar)
    diag_b = multivariate_normal(np.squeeze(np.asarray(transpose(theta_bstar))), inv(Lambda_bstar), 1)[0]
    B = diag(diag_b)
    
    R_star = transpose(U)@B@V
    predicted_values = R_star[nonzero_indices]
    rmse = np.sqrt(MSE(nonzero_values, predicted_values))
    
    if it%10 == 0:
        print("Iter:", it, "RMSE:", rmse)

Iter: 0 RMSE: 1.0426667045732603
Iter: 10 RMSE: 0.946910204861326
Iter: 20 RMSE: 0.9478114012176595
Iter: 30 RMSE: 0.9466188999382511
Iter: 40 RMSE: 0.95458703126033
Iter: 50 RMSE: 0.9528870393775221
Iter: 60 RMSE: 0.9614578651701132
Iter: 70 RMSE: 0.9602421142338486
Iter: 80 RMSE: 0.9468473064722094
Iter: 90 RMSE: 0.9530720311468398


### k = 1、5、10、20、...